<a href="https://colab.research.google.com/github/heungsikyu/data_analysis_study/blob/master/Stock_Prediction_with_KreasLSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np

import plotly.graph_objs as go 
import plotly.offline as py 

import matplotlib.pyplot as plt

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import pandas_datareader as pdr


from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
# from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import datetime

삼성전자 005930.KS 

LG전자 066570.KS

현대차 005380.KS

In [ ]:
df = pdr.get_data_yahoo('005930.KS', adjust_price=True)

In [ ]:
df.head()

In [ ]:
df

주식의 중간 값  작성 

In [ ]:
high_prices = df['High'].values
# high_prices
low_prices = df['Low'].values
mid_price = (high_prices+low_prices)/2

윈도우 만들기 - 

In [ ]:
seq_len = 50
sequence_length = seq_len + 1

result = []

for index in range(len(mid_price) - sequence_length):
  result.append(mid_price[index: index+sequence_length])

result[:5]

Nomarlize Data

In [ ]:
normalized_data = []

for window in result:
    normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
    normalized_data.append(normalized_window)

result = np.array(normalized_data)

# split train and test data
row = int(round(result.shape[0] * 0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1]

x_train.shape, x_test.shape

Build a Model

In [ ]:
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))

model.add(LSTM(64, return_sequences=False))

model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='rmsprop')

model.summary()

Training

In [ ]:
model.fit(x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=10,
    epochs=60)

Prediction

In [ ]:
pred = model.predict(x_test)

fig = plt.figure(facecolor='white', figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(y_test, label='True')
ax.plot(pred, label='Prediction')
ax.legend()
plt.show()

In [ ]:
pred = model.predict(x_test)
pred.shape
predt = pred.T
predT = predt[0]
y_test1 = y_test.astype(np.float32)

In [ ]:
fig = go.Figure()
fig.add_trace(
  go.Scatter(
      # df, x="sepal_width", y="sepal_length",
      # x = df['Date'],
      mode='lines+markers',
      name="test",
      y = y_test1,
  )
)
fig.add_trace(
  go.Scatter(
      # df, x="sepal_width", y="sepal_length",
      # x = df['Date'],
      mode='lines+markers',
      name='Predict',
      y = predT,
      line = dict(color='orange', width=4, dash='dot')
  )
)


In [ ]:
import plotly.express as px

# df = px.data.gapminder().query("continent != 'Asia'") # remove Asia for visibility
df = px.data.gapminder()

df


In [ ]:
fig = px.line(df, x="year", y="lifeExp", color="continent",
              line_group="country", hover_name="country")
fig.show()